In [1]:
import easyocr

from PIL import Image

import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt

import numpy as np

from enum import IntEnum

import cv2
import os
import json

C:\ProgramData\Anaconda3\envs\ocr\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!copy C:\Users\notebook\Downloads\_my_dataset\20221214_hateful_memes\hateful_memes\train.jsonl C:\Users\notebook\Downloads\_my_dataset\20221214_hateful_memes\hateful_memes\train_ocr_no.jsonl
!copy C:\Users\notebook\Downloads\_my_dataset\20221214_hateful_memes\hateful_memes\test.jsonl C:\Users\notebook\Downloads\_my_dataset\20221214_hateful_memes\hateful_memes\test_ocr_no.jsonl

‘Є®ЇЁа®ў ­® д ©«®ў:         1.
‘Є®ЇЁа®ў ­® д ©«®ў:         1.


In [3]:
data_dir = r'C:\Users\notebook\Downloads\_my_dataset\20221214_hateful_memes\hateful_memes\\'

train_path = data_dir + 'train_ocr_no.jsonl'
dev_path = data_dir + 'test_ocr_no.jsonl'

train_data = pd.read_json(train_path, lines=True)
test_data = pd.read_json(dev_path, lines=True)

In [4]:
train_data['path'] = train_data['img'].apply(lambda x: os.path.join(data_dir, x))
train_data

,label,img,path
0,3,img/10910387.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
1,3,img/10747369.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
2,0,img/13996726.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
3,1,img/9987066.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
4,0,img/16702250.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
...,...,...,...
626,3,img/14293988.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
627,3,img/13082683.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
628,0,img/15659398.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
629,0,img/12081951.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...


In [5]:
test_data['path'] = test_data['img'].apply(lambda x: os.path.join(data_dir, x))
test_data

,label,img,path
0,0,img/13037928.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
1,6,img/9664522.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
2,1,img/15448302.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
3,0,img/16702294.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
4,0,img/15244154.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
...,...,...,...
206,1,img/12954537.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
207,6,img/16180208.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
208,0,img/15244043.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...
209,0,img/14993032.jpg,C:\Users\notebook\Downloads\_my_dataset\202212...


In [6]:
class ImageTransformType(IntEnum):
    No = 0
    Gray = 1
    Binary = 2
    BinaryInv = 3
    AdaptiveMean = 4
    AdaptiveMeanInv = 5
    AdaptiveGaussian = 6
    AdaptiveGaussianInv = 7

In [7]:
def recognize(path:list, recognizer_name: str, image_transform: ImageTransformType, thresh: int, maxval: int, 
              block_size:int, c:float) -> list:
    recognized_text = []    
    for i in range(len(path)):
        #print(path[i])
        image = cv2.imread(path[i], cv2.COLOR_BGR2RGB)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
        if image_transform == ImageTransformType.Gray:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
        if image_transform == ImageTransformType.Binary:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            _, image = cv2.threshold(image, thresh, maxval, cv2.THRESH_BINARY)
            
        if image_transform == ImageTransformType.BinaryInv:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            _, image = cv2.threshold(image, thresh, maxval, cv2.THRESH_BINARY_INV)

        if image_transform == ImageTransformType.AdaptiveMean:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)            
            image = cv2.adaptiveThreshold(image, maxval, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, block_size, c)

        if image_transform == ImageTransformType.AdaptiveMeanInv:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)            
            image = cv2.adaptiveThreshold(image, maxval, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, block_size, c)
            
        if image_transform == ImageTransformType.AdaptiveGaussian:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)            
            image = cv2.adaptiveThreshold(image, maxval, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, block_size, c)
            
        if image_transform == ImageTransformType.AdaptiveGaussianInv:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)            
            image = cv2.adaptiveThreshold(image, maxval, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, block_size, c)
            
        if recognizer_name == 'easyocr':            
            recognized_text.append(' '.join(easyocr_reader.readtext(image, detail=0)))
            
        elif recognizer_name == 'pytesseract':
            recognized_text.append(pytesseract.image_to_string(image).replace('\n', ' '))
            
    return recognized_text


In [8]:
easyocr_reader = easyocr.Reader(['ru']) # this needs to run only once to load the model into memory

In [10]:
recognized_text_test = recognize(test_data['path'].values, 
     recognizer_name='easyocr', 
     image_transform=ImageTransformType.No, 
     thresh=0, 
     maxval=0,
     block_size=0,
     c=0)


In [11]:
recognized_text_test

['СЕКОеПУЧШШШЕВ (@АРСЛВО [АШШУЯУЖжВ ВЫЗАОРОВИ^ НЕГ; НЕ ВЫЗДОРОВИО)',
 '[в[езабыть Мами ШиШаВЕДШОВОщеЙ',
 'Она: все , мне больше не нужны мужики! С ЭтОго дня я гордая и независимая! Она, через 2 дня с весенним обострением: 0 а кому сейчас лехко',
 'Измени свою жизнь + Христианство : хорошо хоть в жопу никого не выебали. впрочем, еще не вечер сегодня в 0.05 38',
 'Невидимый кЕ?е  и Хэнк Всадники ЯП Апокалипсиса Муди',
 'КРЫЧАЙ [РАЗ] [Е271(5] ГОМЕЛЬ [Р93] МАГЕЛЁЙ [РЗ1 МАЗЫР БАБРУЙСК (62 ЧАШНЕКЬ (55] ЛЮБАНЬ БАРЫСАЙ (Р67 [4З МВАЦЭВНЧЫ 4ар,]) @#рославль Ярославль',
 'Мы хотим Угомони свой пыл! Я не смогу равенства СТОЛЬКО грабить И насиловать',
 'Я тут тебя с какими-То Да это с мужиками видел работы . . Прости, иногда Да ничего, забываю, что ты зай. проститутка',
 '@О@@дИ @цЕНят',
 'Я МЕНЯ^А ПОАГУЗНИК ГАЕ ЪЫ БЫХА; НАШЕМУ СЫНУ . СхоАИ Я ТЕБЯ ИСКА^. ВЫКИНЬ ЕГО, ПОЖААУЙСТА НЕ ВОПРОС',
 'ЗАМА НИАИ ПИРОГАМИ И отпИзДИ/И НОГАМИ УАРАКАПНП',
 '0 А~ме гц Паша С<а: оп/пе Милая; ну как тебе доказать; ЧТ

In [12]:

dev_rows = pd.read_json(dev_path, lines=True)
with open(data_dir + 'test_ocr_no.jsonl', "w", encoding="utf-8") as rf:
    for i, (_, row) in enumerate(dev_rows.iterrows()):
        dictRow = dict(row)
        dictRow["recognized_text_easyocr"] = recognized_text_test[i]
        rf.write(json.dumps(dictRow) + "\n")
        
        

In [13]:
recognized_text_train = recognize(train_data['path'].values, 
     recognizer_name='easyocr', 
     image_transform=ImageTransformType.No, 
     thresh=0, 
     maxval=0,
     block_size=0,
     c=0)



In [14]:

train_rows = pd.read_json(train_path, lines=True)
with open(data_dir + 'train_ocr_no.jsonl', "w", encoding="utf-8") as rf:
    for i, (_, row) in enumerate(train_rows.iterrows()):
        dictRow = dict(row)
        dictRow["recognized_text_easyocr"] = recognized_text_train[i]
        rf.write(json.dumps(dictRow) + "\n")

In [15]:
train_path_ocr = data_dir + 'train_ocr_no.jsonl'
dev_path_ocr = data_dir + 'test_ocr_no.jsonl'

train_data_ocr = pd.read_json(train_path_ocr, lines=True)
test_data_ocr = pd.read_json(dev_path_ocr, lines=True)

In [16]:
train_data_ocr

,label,img,recognized_text_easyocr
0,3,img/10910387.jpg,ЧТО ДЕЛАТЫ ЕСЛИ НА ВАС НАПАЛ МЕДВЕДЬ А У ВАС П...
1,3,img/10747369.jpg,ТЫКУРШПЫ ШШКШКПДАКУНЮ; ТИК [ЕП /АП_=
2,0,img/13996726.jpg,"ОКАЗАЛОСЬ, МОЯ СЕСТРА ПО НОЧАМ ЗАНИМАЕТСЯ ПРОС..."
3,1,img/9987066.jpg,аШИСШЫаШ @СЮВшЧ& ВИССЗаЮНЧИШСЯ
4,0,img/16702250.jpg,@ЕВУДОЧКУ ЩА ШШЫШВш шп
...,...,...,...
626,3,img/14293988.jpg,Дша Кл@бОШДШИШПШ
627,3,img/13082683.jpg,"СКОЛЬКО ВЕСИТ ИДЕАЛЬНЫЙ ЕВРЕЙ? 0,2 КИЛОГРАММА ..."
628,0,img/15659398.jpg,ТАТАНЕДЕЛИ 1 лмар Обезьяны умеют говорить но м...
629,0,img/12081951.jpg,Э~&ж. ё3 %с Э'Чл _ @(ъ &@ & [яъ(нес (ъ )(е<<< <


In [17]:
test_data_ocr

,label,img,recognized_text_easyocr
0,0,img/13037928.jpg,СЕКОеПУЧШШШЕВ (@АРСЛВО [АШШУЯУЖжВ ВЫЗАОРОВИ^ Н...
1,6,img/9664522.jpg,[в[езабыть Мами ШиШаВЕДШОВОщеЙ
2,1,img/15448302.jpg,"Она: все , мне больше не нужны мужики! С ЭтОго..."
3,0,img/16702294.jpg,Измени свою жизнь + Христианство : хорошо хоть...
4,0,img/15244154.jpg,Невидимый кЕ?е и Хэнк Всадники ЯП Апокалипсис...
...,...,...,...
206,1,img/12954537.jpg,КАК НАПУГАТЬ ЕГО Я БЕРЕМНА КАК НАПУГАТЬ ЕЕ Я л...
207,6,img/16180208.jpg,
208,0,img/15244043.jpg,"Да что может быть хуже, чем несколько детей в ..."
209,0,img/14993032.jpg,КОГДА ПРИ ЧИСТКЕ МАНДАРИНА #Е ЧЧВСТВЧЕШЬ ЗАПАХ...
